# Data Preparation

### Data Loading and Merging

In [1]:
import pandas as pd

# Read the CSV files
attendance = pd.read_csv('data/game_info.csv') #contains the attendancce column
games = pd.read_csv('data/game.csv') #contains game information (which teams, who won, etc.)
stadion_capacities = pd.read_csv('stadion_capacity.csv') #contains stadium capacities and when the stadium was used

# Merge attendance and games on game_id
games = pd.merge(attendance, games, on='game_id', how='inner')

games.drop(columns=['game_date_y'], inplace=True) # drop second game date
games.rename(columns={'game_date_x': 'game_date'}, inplace=True) 
games['game_date'] = pd.to_datetime(games['game_date'])

# Filter for games from 1996-97 season onwards
# NBA season starts in October, so keep games from October 1996 onwards
games_1996 = games[
    ((games['game_date'].dt.year == 1996) & (games['game_date'].dt.month >= 10)) |
    (games['game_date'].dt.year > 1996)
]

# Add a year column
games_1996.loc[:, 'game_year'] = games_1996['game_date'].dt.year

# Save to CSV
# games_1996.to_csv('nba_games_since_1996.csv', index=False)


C:\Users\Martin\AppData\Local\Temp\ipykernel_23328\2912657500.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_1996.loc[:, 'game_year'] = games_1996['game_date'].dt.year


In [2]:
# Create a function to determine if a game is in first or second half of season
def get_season_half(date):
    # NBA season typically starts in October and ends in April
    # If month is October through December, it's first half
    # If month is January through June, it's second half
    if date.month >= 10:
        return 'First Half'
    else:
        return 'Second Half'

# Function to match game with correct arena capacity
def get_arena_capacity(row, stadiums_df):
    game_date = row['game_date']
    team_id = row['team_id_home']
    
    # Get all stadiums for this team
    team_stadiums = stadiums_df[stadiums_df['team_id'] == team_id]
    
    # Find the correct stadium based on date
    for _, stadium in team_stadiums.iterrows():
        start_year = stadium['start_year']
        end_year = stadium['end_year']
        
        # Handle the mid-year season transition
        if game_date.month >= 10:
            game_year = game_date.year + 1
        else:
            game_year = game_date.year
            
        if start_year <= game_year <= end_year:
            return stadium['capacity']
    
    return None

In [3]:
# Add season half (First or Second Half)
games_1996.loc[:, 'season_half'] = games_1996['game_date'].apply(get_season_half)

# Get arena capacity for each game
games_1996.loc[:, 'arena_capacity'] = games_1996.apply(lambda x: get_arena_capacity(x, stadion_capacities), axis=1)

# Calculate occupancy rate
games_1996.loc[:, 'occupancy_rate'] = (games_1996['attendance'] / games_1996['arena_capacity']) * 100

# Round occupancy rate to 2 decimal places
# occupancy_df['occupancy_rate'] = occupancy_df['occupancy_rate'].round(2)

# Sort by date
# occupancy_df = occupancy_df.sort_values('game_date')

C:\Users\Martin\AppData\Local\Temp\ipykernel_23328\3256368380.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_1996.loc[:, 'season_half'] = games_1996['game_date'].apply(get_season_half)
C:\Users\Martin\AppData\Local\Temp\ipykernel_23328\3256368380.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_1996.loc[:, 'arena_capacity'] = games_1996.apply(lambda x: get_arena_capacity(x, stadion_capacities), axis=1)
C:\Users\Martin\AppData\Local\Temp\ipykernel_23328\3256368380.py:8: SettingWithCopyWar

In [4]:
# Move "arena_capacity" and "occupancy_rate" right after "attendance" and disregard columns like fg % -> shorten the dataframe
new_order = [
    "game_id", "game_date", "attendance", 
    "arena_capacity", "occupancy_rate", 
    "game_time", "game_year", "season_id", "pts_home", 'pts_away', "plus_minus_home",  "plus_minus_away",
    "team_id_home", "team_abbreviation_home", "matchup_home", "wl_home",
    "team_id_away", "team_abbreviation_away",
    "season_type", "season_half"
]

# Reorder DataFrame


master_short_data = games_1996[new_order]
master_short_data.to_csv('Master_Short_Data.csv', index=False)

In [5]:
games_1996.to_csv('Master_Full_Data.csv', index=False)

In [34]:
# import pandas as pd

# # Load the dataset
# df = pd.read_csv('Master_Short_Data.csv')

# # Remove rows where game_year is 2020
# df = df[df['game_year'] != 2020]
# df = df[df['game_year'] != 2021]


# # Save the cleaned dataset
# df.to_csv('Master_Short_Data.csv', index=False)

### Filtering and Cleaning

In [6]:
Master = master_short_data

# Daten laden
df = pd.read_csv('Master_Short_Data.csv')
df['game_date'] = pd.to_datetime(df['game_date'])

# Extract date-based features
df['game_year'] = df['game_date'].dt.year
df['game_month'] = df['game_date'].dt.month
df['game_dayofweek'] = df['game_date'].dt.dayofweek  # Monday=0, Sunday=6
df['game_hour'] = pd.to_datetime(df['game_time'], errors='coerce').dt.hour


df = df.dropna(subset=['arena_capacity'])
df = df.dropna(subset=['attendance'])

# Liste aller Teams, die mindestens einmal als Heimteam vorkommen
home_teams = df['team_abbreviation_home'].unique()

# Nur Einträge behalten, bei denen das Auswärtsteam auch in den Heimteams vorkommt
df = df[df['team_abbreviation_away'].isin(home_teams)]



df.to_csv('Master_Short_Data.csv', index=False) 

C:\Users\Martin\AppData\Local\Temp\ipykernel_23328\833280829.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['game_hour'] = pd.to_datetime(df['game_time'], errors='coerce').dt.hour


In [7]:
df = pd.read_csv('Master_Short_Data.csv')

# drop all rows between 2020 march and 2021 september
df = df[(df['game_date'] < '2020-03-01') | (df['game_date'] > '2021-10-15')]

df.to_csv('Master_Short_Data.csv', index=False) 

### Adding New Features

### Adding Records of Games to Master Short 

In [8]:
def calculate_team_records(df):
    # Create copy to avoid modifying original
    df = df.copy()
    
    # Sort by season and date to ensure correct order
    # df = df.sort_values(['season_id', 'game_date'])
    
    # Initialize new columns
    df['home_wins'] = 0
    df['home_losses'] = 0
    df['home_win_streak'] = 0
    df['away_wins'] = 0
    df['away_losses'] = 0
    df['away_win_streak'] = 0
    
    # Dictionary to store team records for each season
    # Format: {season_id: {team_id: {'wins': x, 'losses': y, 'streak': z}}}
    season_records = {}
    
    # Process each game
    for idx, row in df.iterrows():
        season = row['season_id']
        home_team = row['team_id_home']
        away_team = row['team_id_away']
        
        # Initialize season records if needed
        if season not in season_records:
            season_records[season] = {}
            
        # Get or initialize home team records
        if home_team not in season_records[season]:
            home_record = {'wins': 0, 'losses': 0, 'streak': 0}
        else:
            home_record = season_records[season][home_team].copy()
            
        # Get or initialize away team records
        if away_team not in season_records[season]:
            away_record = {'wins': 0, 'losses': 0, 'streak': 0}
        else:
            away_record = season_records[season][away_team].copy()
        
        # Add result of current game
        if row['wl_home'] == 'W':
            home_record['wins'] += 1
            away_record['losses'] += 1
            home_record['streak'] = max(1, home_record['streak'] + 1)
            away_record['streak'] = min(-1, away_record['streak'] - 1)
        else:
            home_record['losses'] += 1
            away_record['wins'] += 1
            home_record['streak'] = min(-1, home_record['streak'] - 1)
            away_record['streak'] = max(1, away_record['streak'] + 1)
        
        # Store current records including this game
        df.at[idx, 'home_wins'] = home_record['wins']
        df.at[idx, 'home_losses'] = home_record['losses']
        df.at[idx, 'home_win_streak'] = home_record['streak']
        df.at[idx, 'away_wins'] = away_record['wins']
        df.at[idx, 'away_losses'] = away_record['losses']
        df.at[idx, 'away_win_streak'] = away_record['streak']
        
        # Store updated records for next games
        season_records[season][home_team] = home_record
        season_records[season][away_team] = away_record
    
    return df

In [9]:
calculate_team_records(df).to_csv('Master_Short_Data.csv', index=False)

### Adding Rivalries to Dataset

In [10]:
import pandas as pd

# Load Master Short Data and rivalries
master_df = pd.read_csv('Master_Short_Data.csv')
rivalries_df = pd.read_csv('rivalries.csv')

# Prepare set of rivalry pairs (both directions)
rival_pairs = set()
for _, row in rivalries_df.iterrows():
    t1 = row['team_id']
    t2 = row['rival_team_id']
    rival_pairs.add((t1, t2))
    rival_pairs.add((t2, t1))

def is_rival_game(row):
    return int((row['team_id_home'], row['team_id_away']) in rival_pairs)

# Add rival_game column (1 = yes, 0 = no)
master_df['rival_game'] = master_df.apply(is_rival_game, axis=1)

# Save updated dataframe
master_df.to_csv('Master_Short_Data.csv', index=False)

### Adding defending champions

In [11]:
import pandas as pd

# Load champions data if not already loaded
champions_df = pd.read_csv('champions.csv')

# Function to get defending champion for a given team, year, and season_half
def get_defending_champ(team_id, game_year, season_half):
    if season_half == 'Second Half':
        champ_year = game_year - 1
    else:
        champ_year = game_year
    champ_row = champions_df[champions_df['year'] == champ_year]
    if not champ_row.empty and champ_row.iloc[0]['team_id'] == team_id:
        return 1
    return 0

# Add defending champion columns for home and away teams
master_df['defending_champ_home'] = master_df.apply(
    lambda row: get_defending_champ(row['team_id_home'], row['game_year'], row['season_half']), axis=1
)
master_df['defending_champ_away'] = master_df.apply(
    lambda row: get_defending_champ(row['team_id_away'], row['game_year'], row['season_half']), axis=1
)

# Save updated dataframe
master_df.to_csv('Master_Short_Data.csv', index=False)

### Adding largest leads and lead_changes

In [12]:
import pandas as pd

# Hauptdatensatz laden
master_df = pd.read_csv("Master_Short_Data.csv")

# Zusätzliche Statistikdaten laden
other_stats = pd.read_csv("Data/other_stats.csv")

# Relevante Spalten aus other_stats wählen
other_stats_subset = other_stats[[
    'game_id', 'largest_lead_home', 'largest_lead_away', 'lead_changes', 'times_tied'
]]

# Achtung: game_id muss in beiden Datensätzen vorhanden sein
# Prüfen, ob 'game_id' auch im Master-Datensatz korrekt existiert:
if 'game_id' not in master_df.columns:
    print("⚠️ 'game_id' fehlt im Master-Satz, bitte prüfen!")

# Merge anhand der game_id
merged_df = master_df.merge(other_stats_subset, on='game_id', how='left')

# Ergebnis prüfen
print(merged_df[['game_id', 'largest_lead_home', 'largest_lead_away', 'lead_changes']].head())

# Neues CSV speichern (optional)
merged_df.to_csv("Master_Short_Data.csv", index=False)


    game_id  largest_lead_home  largest_lead_away  lead_changes
0  29600012                1.0               19.0           4.0
1  29600005                0.0               16.0           0.0
2  29600002               20.0                2.0           1.0
3  29600007               10.0                9.0           7.0
4  29600013               12.0                6.0           5.0


### Adding MVPs to Dataset

In [13]:
import pandas as pd

# Load Master Short Data and MVPs
master_df = pd.read_csv('Master_Short_Data.csv')
mvps_df = pd.read_csv('mvps.csv')

# Prepare MVP mapping for both season splits
mvp_map = {}
for _, row in mvps_df.iterrows():
    mvp_team = row['team_id']
    mvp_year = row['year']
    # Assign MVP to previous year + Second Half and current year + First Half
    mvp_map[(mvp_year - 1, 'Second Half', mvp_team)] = 1
    mvp_map[(mvp_year, 'First Half', mvp_team)] = 1

def has_mvp(row):
    key = (row['game_year'], row['season_half'], row['team_id_home'])
    return mvp_map.get(key, 0)

# Add MVP column for home team
master_df['home_mvp'] = master_df.apply(has_mvp, axis=1)

# Add MVP column for away team
def has_mvp_away(row):
    key = (row['game_year'], row['season_half'], row['team_id_away'])
    return mvp_map.get(key, 0)

master_df['away_mvp'] = master_df.apply(has_mvp_away, axis=1)

# Save updated dataframe
master_df.to_csv('Master_Short_Data.csv', index=False)

### Adding attendance of the previous season to Dataset

In [14]:
import pandas as pd
import numpy as np

# Ensure game_date is datetime
master_df['game_date'] = pd.to_datetime(master_df['game_date'])

def get_prev_season_avg(row, df):
    team = row['team_id_home']
    season = row['game_year']
    half = row['season_half']
    
    if half == 'Second Half':
        # Previous season: previous year Second Half + current year First Half
        mask = (
            ((df['team_id_home'] == team) & 
             (((df['game_year'] == season - 1) & (df['season_half'] == 'Second Half')) |
              ((df['game_year'] == season) & (df['season_half'] == 'First Half'))))
        )
    else:  # First Half
        # Previous season: two years back Second Half + one year back First Half
        mask = (
            ((df['team_id_home'] == team) & 
             (((df['game_year'] == season - 2) & (df['season_half'] == 'Second Half')) |
              ((df['game_year'] == season - 1) & (df['season_half'] == 'First Half'))))
        )
    prev_games = df.loc[mask, 'attendance']
    if len(prev_games) == 0:
        return np.nan
    return prev_games.mean()

master_df['attendance_prev_season_avg'] = master_df.apply(lambda row: get_prev_season_avg(row, master_df), axis=1)

# Save to CSV
master_df.to_csv('Master_Short_Data.csv', index=False)

### Adding laged attendance (two games) to Dataset

In [15]:
# Add lag features for attendance (previous 1 and 2 games for each home team)
df = pd.read_csv('Master_Short_Data.csv')
df = df.sort_values(['team_id_home', 'game_date'])

df['attendance_lag1'] = df.groupby('team_id_home')['attendance'].shift(1)
df['attendance_lag2'] = df.groupby('team_id_home')['attendance'].shift(2)


df = df.sort_values(['game_date'])

df.to_csv('Master_Short_Data.csv', index=False)

### Adding Sold Out Variable (>= 0.99 occuoancy)

In [16]:
import pandas as pd

# Daten einlesen
df = pd.read_csv('Master_Short_Data.csv')

# Neue Spalte 'sold_out' erstellen: 1 = ausverkauft, 0 = nicht ausverkauft
df['sold_out'] = (df['attendance'] >= (df['arena_capacity']*0.99)).astype(int)

# Ergebnis prüfen
print(df[['attendance', 'arena_capacity', 'sold_out']].head(10))

# Optional: CSV mit neuer Spalte abspeichern
df.to_csv('Master_Short_Data.csv', index=False)


   attendance  arena_capacity  sold_out
0     18109.0         18798.0         0
1     19193.0         18422.0         1
2     20444.0         20478.0         1
3     16104.0         17171.0         0
4     15593.0         19596.0         0
5     17248.0         15077.0         1
6     18624.0         19156.0         0
7     16285.0         15676.0         1
8     18316.0         20049.0         0
9     21454.0         22076.0         0


## Analysis

In [45]:
# Calculate number of games per year with occupancy rate less than 50%
games_below_50 = master_df[master_df['occupancy_rate'] < 50]
games_below_50_per_year = games_below_50.groupby('game_year').size().reset_index(name='games_below_50pct')

print(games_below_50_per_year)

    game_year  games_below_50pct
0        1996                 11
1        1997                 34
2        1998                 43
3        1999                 39
4        2000                  3
5        2001                 17
6        2002                 37
7        2003                 12
8        2004                  4
9        2005                 22
10       2006                  8
11       2007                 29
12       2008                 25
13       2009                 21
14       2010                 29
15       2011                  5
16       2012                 25
17       2013                 25
18       2014                 27
19       2015                 19
20       2016                 22
21       2017                  5
22       2020                  2
23       2021                459
24       2022                 10


In [48]:
# Calculate number of games with occupancy rate of 100% for each home team and year
pd.options.display.max_rows = 4000


games_over_100 = master_df[master_df['occupancy_rate'] >= 100]
over_100_counts = games_over_100.groupby(['game_year', 'team_abbreviation_home']).size().reset_index(name='games_over_100pct')

# Create all combinations of years and teams to ensure completeness
all_years = master_df['game_year'].unique()
all_teams = pd.DataFrame({'team_abbreviation_home': home_teams})
all_combinations = pd.MultiIndex.from_product([all_years, home_teams], names=['game_year', 'team_abbreviation_home']).to_frame(index=False)

# Merge and fill missing with zero
over_100_counts = all_combinations.merge(over_100_counts, on=['game_year', 'team_abbreviation_home'], how='left').fillna(0)
over_100_counts['games_over_100pct'] = over_100_counts['games_over_100pct'].astype(int)

# Sort by year and number of games descending
over_100_counts = over_100_counts.sort_values(['game_year', 'games_over_100pct'], ascending=[True, False])

print(over_100_counts.head(3000))

      game_year team_abbreviation_home  games_over_100pct
11         1996                    UTA                 17
5          1996                    HOU                 13
16         1996                    CHI                 13
17         1996                    CHH                 12
19         1996                    PHX                 12
23         1996                    POR                 12
25         1996                    SAC                 12
12         1996                    ORL                 11
0          1996                    LAL                  6
22         1996                    SAS                  4
2          1996                    NJN                  3
18         1996                    DAL                  3
9          1996                    PHI                  2
15         1996                    ATL                  2
6          1996                    VAN                  1
10         1996                    MIN                  1
20         199

In [47]:
# Calculate number of games with occupancy rate of 100% for each home team
games_over_100 = master_df[master_df['occupancy_rate'] >= 100]
over_100_counts = games_over_100.groupby('team_abbreviation_home').size().reset_index(name='games_over_100pct')

# Ensure all teams are included, fill missing with zero
all_teams = pd.DataFrame({'team_abbreviation_home': home_teams})
over_100_counts = all_teams.merge(over_100_counts, on='team_abbreviation_home', how='left').fillna(0)
over_100_counts['games_over_100pct'] = over_100_counts['games_over_100pct'].astype(int)

# Sort by number of games descending
over_100_counts = over_100_counts.sort_values('games_over_100pct', ascending=False)

print(over_100_counts)

   team_abbreviation_home  games_over_100pct
0                     LAL                852
18                    DAL                840
11                    UTA                816
16                    CHI                732
19                    PHX                717
1                     MIA                686
23                    POR                653
22                    SAS                638
24                    CLE                579
5                     HOU                543
12                    ORL                536
27                    TOR                516
4                     GSW                512
25                    SAC                490
33                    OKC                431
15                    ATL                376
3                     DET                372
13                    IND                321
20                    MIL                287
9                     PHI                252
14                    LAC                246
21        

In [ ]:

# Calculate number of games with occupancy rate over 100% for each home team
games_over_100 = master_df[master_df['occupancy_rate'] == 100]
over_100_counts = games_over_100.groupby('team_abbreviation_home').size().reset_index(name='games_over_100pct')

# Sort by number of games descending
over_100_counts = over_100_counts.sort_values('games_over_100pct', ascending=False)

print(over_100_counts)

   team_abbreviation_home  games_over_100pct
8                     GSW                452
22                    SAC                441
24                    TOR                432
18                    OKC                392
13                    MIA                350
7                     DET                338
17                    NYK                239
9                     HOU                139
25                    UTA                132
1                     BOS                122
0                     BKN                116
23                    SAS                110
12                    MEM                103
4                     CLE                 98
19                    ORL                 93
11                    LAC                 73
21                    POR                 61
3                     CHH                 60
14                    MIL                 59
26                    WAS                 54
6                     DEN                 54
15        

#### Wie viele Spiele sind Ausverkauft

In [44]:
import pandas as pd

# Daten laden
df = pd.read_csv("Master_Short_Data.csv")

df['game_date'] = pd.to_datetime(df['game_date'])
df = df[(df['game_date'].dt.year >= 2023) & (df['game_date'].dt.year <= 2024)]


# Gesamtzahl der Spiele
total_games = len(df)

# Anzahl ausverkaufter Spiele
sold_out_games = df['sold_out'].sum()

# Prozentualer Anteil
sold_out_percentage = sold_out_games / total_games * 100

print(f"Anzahl ausverkaufter Spiele: {sold_out_games}")
print(f"Gesamtzahl Spiele: {total_games}")
print(f"Anteil ausverkauft: {sold_out_percentage:.2f}%")

### Calculate Games Played per Season for Teams

In [41]:
def get_team_games_per_season(df, team_abbr='CHI'):
    # Create a mask for the selected team (both home and away games)
    team_games = df[
        (df['team_abbreviation_home'] == team_abbr) | 
        (df['team_abbreviation_away'] == team_abbr)
    ]
    
    # Group by season_id and season_type and count games
    games_count = team_games.groupby(['season_id', 'season_type']).size().reset_index(name='games_played')
    
    # Sort by season_id
    games_count = games_count.sort_values('season_id')
    
    # Add season start year (season_id minus 20000)
    games_count['season_start'] = games_count['season_id'].apply(lambda x: str(x - 20000))
    
    # Reorder columns
    games_count = games_count[['season_start', 'season_id', 'season_type', 'games_played']]
    
    return games_count

# Calculate games for Chicago Bulls
chi_games = get_team_games_per_season(games, 'CHI')
print("\nChicago Bulls Games per Season:")
print(chi_games.to_string(index=False))

# Save to CSV for further analysis
chi_games.to_csv('chi_games_per_season.csv', index=False)

### Calculate Top 5 Attendance Games per Team per Season

In [42]:
master = pd.read_csv('Master_Short_Data.csv')

def analyze_top_attendance(df, n=5):
    # Create copy of dataframe with only needed columns
    df_attendance = df[['season_id', 'team_id_home', 'team_abbreviation_home', 'attendance', 'arena_capacity', 'season_type']]
    
    # Filter out preseason and all-star games, keep regular season and playoffs
    df_attendance = df_attendance[
        (df_attendance['season_type'].isin(['Regular Season', 'Playoffs'])) & 
        (df_attendance['attendance'].notna())
    ]
    
    # Group by season and team
    def get_top_n_details(group):
        # Initialize empty result dictionary
        result = {
            'games_in_top_5': 0,
            'total_games': len(group)
        }
        
        # Add placeholder values for all potential columns
        for i in range(1, n+1):
            result[f'attendance_{i}'] = 0
            result[f'count_{i}'] = 0
            
        # If group is empty or has no valid attendance values, return empty result
        if len(group) == 0 or group['attendance'].isna().all():
            return pd.Series(result)
            
        # Get top N attendances and their counts
        top_n_values = group['attendance'].value_counts().nlargest(n)
        
        if len(top_n_values) > 0:
            # Update games_in_top_5 count
            result['games_in_top_5'] = (group['attendance'] >= top_n_values.iloc[-1]).sum()
            
            # Add each attendance value and its count
            for i, (attendance, count) in enumerate(top_n_values.items(), 1):
                result[f'attendance_{i}'] = attendance
                result[f'count_{i}'] = count
                
        return pd.Series(result)
    
    # Calculate results for each team-season
    results = df_attendance.groupby(['season_id', 'team_id_home', 'team_abbreviation_home']).apply(get_top_n_details).reset_index()
    
    # Add season start year for easier reading
    results['season_start'] = results['season_id'].apply(lambda x: str(x - 20000))
    
    # Calculate percentage of games in top 5
    results['pct_games_top_5'] = (results['games_in_top_5'] / results['total_games'] * 100).round(1)
    
    # Reorder columns
    cols = ['season_start', 'season_id', 'team_abbreviation_home']
    for i in range(1, n+1):
        cols.extend([f'attendance_{i}', f'count_{i}'])
    cols.extend(['games_in_top_5', 'total_games', 'pct_games_top_5'])
    results = results[cols]
    
    # Rename team abbreviation column
    results = results.rename(columns={'team_abbreviation_home': 'team_abbr'})
    
    # Sort by season and team
    results = results.sort_values(['season_id', 'team_abbr'])
    
    return results

# Calculate results for top 5 attendance games
attendance_analysis = analyze_top_attendance(master, 5)

print("\nTop 5 attendance details per team per season (including playoffs):")
print(attendance_analysis.to_string(index=False))

# Save to CSV
attendance_analysis.to_csv('top_5_attendance_details_with_playoffs.csv', index=False)

# Display summary statistics grouped by team
summary = attendance_analysis.groupby('team_abbr').agg({
    'attendance_1': 'mean',
    'count_1': 'mean',
    'games_in_top_5': 'mean',
    'total_games': 'mean',
    'pct_games_top_5': 'mean'
}).sort_values('pct_games_top_5', ascending=False)

print("\nAverage statistics per team:")
print(summary.to_string())